# YandexGPT в DataSphere

Вы можете обращаться к модели YandexGPT из ноутбуков DataSphere через API. 

**YandexGPT API находится на стадии Preview.**

## Содержание

1. [Установите зависимости](#section-id1).
2. [Настройте подключение к облаку](#section-id2).
3. [Обратитесь к модели](#section-id3).
4. [Примеры промптов для YandexGPT](#section-id4).

<a id='section-id1'></a>
## Установите зависимости 

После выполнения ячейки перезагрузите ядро (**Kernel -> Reset Kernel**).После выполнения ячейки перезагрузите ядро (**Kernel -> Reset Kernel**).

In [ ]:
# Устанавливаем пакеты, необходимые для обращения к YandexGPT
%pip uninstall jwt
%pip install PyJWT -U

In [ ]:
import requests
import json
import time
import jwt
import os

<a id='section-id2'></a>
## Настройте подключение к облаку

Чтобы обратиться к API YandexGPT, вам потребуется сервисный аккаунт с ролью `ai.languageModels.user` в облаке.
1. Создайте сервисный аккаунт, как описано в [инструкции](https://cloud.yandex.ru/docs/iam/operations/sa/create).
2. В ячейке ниже укажите идентификатор сервисного аккаунта.

In [ ]:
service_account_id = "<идентификатор_сервисного_аккаунта>"

3. [Создайте](https://cloud.yandex.ru/docs/iam/operations/authorized-key/create) авторизованный ключ для сервисного аккаунта. 
4. Сохраните значение ключа в секрете `private-key`. [Как создать секрет](https://cloud.yandex.ru/docs/datasphere/operations/data/secrets).
5. Идентификатор ключа укажите в ячейке ниже.

In [ ]:
key_id = "<идентификатор_ключа>"
private_key = os.environ['private-key']

7. Получите IAM-токен для сервисного аккаунта.

In [ ]:
now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
token = x['iamToken']

<a id='section-id3'></a>
## Обратитесь к модели

В Yandex Cloud доступны модели YandexGPT и YandexGPT Lite. Обращение к моделям происходит по URI. Дополнительная информация о моделях YandexGPT доступна в [документации](https://yandex.cloud/ru/docs/foundation-models/concepts/yandexgpt/models).

In [ ]:
# Адрес для обращения к модели 

url = 'https://llm.api.cloud.yandex.net/foundationModels/v1/completion'

data = {}

# Указываем URI модели. Замените <идентификатор_каталога> своим значением
data['modelUri'] = 'gpt://<идентификатор_каталога>/yandexgpt-lite'

# Настраиваем дополнительные параметры модели
data['completionOptions'] = {'stream': False,
                             'temperature': 0.3,
                             'maxTokens': 1000}

# Указываем контекст для модели
data['messages'] = [
    {
        "role": "system",
        "text": "Ты — рекрутер в указанной компании. \
        Имитируй собеседование на работу для указанной должности,\
        задавая вопросы, как будто ты потенциальный работодатель.\
        Твоя задача — определить технические навыки кандидата. \
        Сгенерируй вопросы для интервью с потенциальным кандидатом."
    }, 
    {
        "role": "user",
        "text": "Компания: Яндекс. Должность: бэкенд-разработчик."
    }
]

# Получаем ответ модели
response = requests.post(url, headers={'Authorization': 'Bearer ' + token}, json = data).json()

In [ ]:
response

#### Примеры промтов для решения различных задач с помощью YandexGPT доступны в [библиотеке промтов YandexGPT API](https://yandex.cloud/ru/docs/foundation-models/prompts/yandexgpt/)

### Краткий пересказ текстов, хранящихся в объектном хранилище 

Чтобы обращаться к данным в бакете объектного хранилища, [настройте коннектор S3](https://cloud.yandex.ru/docs/datasphere/operations/data/connect-to-s3).
После того, как коннектор S3 будет активирован в проекте, укажите путь к файлу в бакете в ячейке ниже. 

In [ ]:
s3_path = "/home/jupyter/mnt/s3/texts-for-gpt"
files = os.listdir(s3_path)
files

In [ ]:
# Замените <идентификатор_каталога> своим значением
def request(data):
    url = 'https://llm.api.cloud.yandex.net/foundationModels/v1/completion'
    response = requests.post(url, headers={'Authorization': 'Bearer ' + token, 
                                           'X-folder-id': '<идентификатор_каталога>'}, json = data).json()
    return response

In [ ]:
data = {}

# Указываем URI модели
data['modelUri'] = 'gpt://<идентификатор_каталога>/summarization'

# Настраиваем дополнительные параметры модели
data['completionOptions'] = {'stream': False,
                             'temperature': 0.1,
                             'maxTokens': 2000} 

In [ ]:
# Для каждого файла в бакете делаем обращение к модели и возвращаем результат

for file_name in files:
    with open(os.path.join(s3_path, file_name), 'r') as file:
        data['messages'] = [{
            "role": "user",
            "text": file.read().rstrip()
        }]
    response = request(data)
    print("Пересказ текста для файла ", file_name, ": \n")
    print(response['result']['alternatives'][0]['text'], "\n\n")
    time.sleep(10)

### Обработка файлов, хранящихся на Яндекс Диске

1. Установите необходимые зависимости.

In [ ]:
%pip install requests 

In [ ]:
import requests
from urllib.parse import urlencode
from io import BytesIO
from zipfile import ZipFile

2. Настройте подключение к Яндекс Диску по [инструкции](https://cloud.yandex.ru/docs/datasphere/operations/data/connect-to-ya-disk).

In [ ]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = '<ссылка_на_файл_на_Яндекс_Диске>'

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
response = requests.get(download_url)

dist_path = './'
zipfile = ZipFile(BytesIO(response.content))
zipfile.extractall(path=dist_path)

In [ ]:
path_files = "gpt_demo" #Укажите название папки, скачанной с Яндекс Диска
files = os.listdir(path_files)
files

In [ ]:
data = {}

# Указываем тип модели
#'gpt://<идентификатор_каталога>/summarization'
data['modelUri'] = 'gpt://<идентификатор_каталога>/summarization'

# Настраиваем дополнительные параметры модели
data['completionOptions'] = {'stream': False,
                             'temperature': 0.1,
                             'maxTokens': 2000}


In [ ]:
# Для каждого файла, скачанного с Яндекс Диска, делаем обращение к модели и возвращаем результат

for file_name in files:
    with open(os.path.join(path_files, file_name), 'r') as file:
        data['messages'] = [{
            "role": "user",
            "text": file.read().rstrip()
        }]
    response = request(data)
    print("Пересказ текста для файла ", file_name, ": \n")
    print(response['result'], "\n\n")
    time.sleep(10)